# Re-implementation of AudioLM

In the following notebook, we are going to introduce a reimplementation of the **AudioLM** network, proposed in the paper *"AudioLM: a Language Modeling Approach to Audio Generation"* (https://arxiv.org/abs/2209.03143).

AudioLM is a state-of-the-art framework built in order to **generate high-quality audio**, while dealing with **long-term consistency**. Trained on a large corpora of audio data, AudioLM is able to provide **natural and coherent audio continuations**, given short initial prompts. The network is also able to **maintain speaker identity**, finding a good trade-off between audio quality and semantical coherence. AudioLM is also able to provide good quality musical continuation from a short prompt, but this will not be discussed in this notebook.


## Import and settings

In [ ]:
# setup for Colab

try:
    import scripts.hubertKM
    print("Working on local")
except ImportError:
    from google.colab import drive
    import os
    import sys
    
    print("Working on colab")

    drive.mount('/content/drive') 

    audio_lm_path = '/content/drive/My Drive/AudioLM'

    # Create the folder if it does not exist
    if not os.path.exists(audio_lm_path):
        os.makedirs(audio_lm_path)
        print(f"Folder created at: {audio_lm_path}")
    else:
        print(f"Folder already exists at: {audio_lm_path}")

    sys.path.append(audio_lm_path)

    %cd {audio_lm_path}
    !git clone https://github.com/GitGab-dev/NN_Project-AudioLM
    %cd NN_Project-AudioLM
    !pip install -r requirements.txt

In [ ]:
# basic import
import torch
import torchaudio
import numpy as np
import random

# data imports
from scripts.hubertKM import SemanticTokenizer
from scripts.SoundStream import soundstream_16khz, audio_to_tokens, tokens_to_audio
from IPython.display import Audio, display
from scripts.data import store_from_librilight, getSingleModelDataLoaders, prepare_single_audio

# model imports
from scripts.TransformerModel import createSingleModel, generate_new_sequence

# trainer imports
from scripts.modelTrainer import setSingleTrainer, modelFit

In [2]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # If using CUDA
np.random.seed(seed)
random.seed(seed)

## Converting Audio Data into Tokens

The most important novelty provided by AudioLM is the usage of a **mixed tokenization approach**, which has never been seen in other Language Modeling competitors. As shown below, we have two tokenization processes that can proceed in parallel.

<center><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/1.png"/></center>

In order to keep informations regarding language syntax and semantic content in speech, the audio waveform is passed through a **w2v-BERT** model that, combined with a K-Means quantizer, returns a set of **Semantic tokens**.

On the other hand, the network needs also to maintain informations about the acoustic features of the audio, in particular pronunciation and speaker identity. In order to do so, the audio waveform is passed through a pretrained audio codec, **SoundStream**, which is able to build an internal hierarchical representation of the audio. Through those representations, called **Acoustic tokens**, the audio is divided into different components, going from the most basic structural audio features (defined as **Coarse Acoustic tokens**) to the fine acoustic details (defined as **Fine Acoustic tokens**).

By modeling both semantic and acoustic tokens within the same framework, the semantic tokens would ensure long-term consistency, while the acoustic tokens would ensure high-quality audio synthesis.

<center><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/2.png"/></center>


### Semantic tokens with w2v-BERT-like model

In the original paper, the semantic tokens are computed through **w2v-BERT**, a recent model trained on learning **self-supervised audio representations**. When trained on large speech corpora, w2v-BERT learns to map the input audio waveform to a rich set of linguistic features.

<center><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/w2vbert.png"/></center>

The model is based on the usage of **Conformer Blocks**, a variation of Transformer blocks augmented with convolution, and during training uses a combination of two self-supervised objectives: a **masked language modeling(MLM) loss** and a **contrastive loss**. Even though this model can be fine-tuned for different tasks, **AudioLM leverages only on the pre-trained version**, using the Context Vectors of as specific MLM Layer as embeddings.
Finally the embeddings are passed through a **K-Means quantizer**, that would simply convert the embeddings into centroid indices, which will be used as Semantic Tokens. During training, the paper propose a value of K=1024 clusters for the quantizer, choosing as Context Vectors the output of the 7th MLM layer normalized.

Regarding our implementation, since **w2v-BERT is a closed-source project**, we opted for a **HuBERT** model as an alternative. We decided to work with a **pretrained version of HuBERT-Base and K-Means** quantizer provided by Fairseq, at https://github.com/facebookresearch/fairseq/blob/main/examples/hubert. In the following model, we use K=500 clusters and the output is taken from the 9th layer of HuBERT.

In [ ]:
w2vBERT = SemanticTokenizer("facebook/hubert-base-ls960","./KmWeights/hubert_base_ls960_L9_km500.bin") 

In [ ]:
example_audio = "exampleAudio.flac"
waveform, sampleRate = torchaudio.load(example_audio)

with torch.no_grad():
    semanticTokens, embeddings = w2vBERT(waveform)

print("Embeddings taken from HuBERT\n")
print(embeddings)
print("\nSemantic Tokens\n")
print(semanticTokens[:30],"...")

### Acoustic tokens with SoundStream codec model

The original framework computes the acoustic tokens using **SoundStream**, a state-of-the-art **neural audio codec** that, through the usage of a convolutional enconder, is able to **map the input audio into a new lower-sampled version**. This allows the model to maintain information about acoustic features, while keeping the audio quality as much as unaffected as possible.

<center><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/soundstream.png"/></center>

The model follows a **common Encoder-Decoder structure**, while the middle part, a **Residual Vector Quantizer**, is responsible to catch all the **meaningful acoustic aspects** of the input waveform, classifing them into a hierarchical order. The codec achieves high quality by being trained end-to-end with a combination of **reconstruction and adversarial losses**.

For AudioLM, the **acoustic tokens are the output embeddings** obtained by the RVQ component, in quantizer order. As already mentioned, the first quantizers capture elementary acoustics, while the last quantizers will encapsulates more fine audio details. In the paper, they proposed using 12 quantizer layers (where the first $Q' = 4$ are used for Coarse tokens), along with a codebook of 1024 and 4 encoder/decoder blocks.

In our implementation, we used a **pretrained SoundStream model** implementation to reduce training time, provided by https://github.com/kaiidams/soundstream-pytorch. In this model version, only **the number of quantizers is reduced** to 8, where 3 of them are dedicated to Coarse tokens.

In [5]:
soundStream = soundstream_16khz()

In [ ]:
example_audio = "exampleAudio.flac"
waveform, sampleRate = torchaudio.load(example_audio)

coarse, fine = audio_to_tokens(waveform, soundStream)

print("Coarse Tokens:")
print(coarse)

print("\nFine Tokens:")
print(fine)

reconstructedWaveform = tokens_to_audio(coarse, fine, soundStream)

print("\nOriginal Audio:\n")
display(Audio(waveform, rate = sampleRate))

print("\nReconstructed Audio:\n")
display(Audio(reconstructedWaveform, rate = sampleRate))

### Token Dataset creation

Now that we have defined models for both the Semantic and Acoustic tokenization, we can proceed with **converting audio data into tokens**. We tried **performing this operation separately** from the AudioLM training, in order to allievate computational costs.

The data used for the model comes from **Libri-Light** (at https://github.com/facebookresearch/libri-light), an high-quality collection of audio-books narrated by different speakers. The dataset already comes with a nice separation in training data and finetuning data. 

In the original paper, they used *unlab-60k*, a combined version of small, medium and big dataset, for a total of about **60000 hours** worth of recording. Since we do not have any computational mean allowing us to train a network on such amount of data, we tried using the **fine-tuning version**, that is worth about **10 hours** of audio data.

Our implementation takes in input the data from **Libri-Light Limited Torch dataset** (refer to https://pytorch.org/audio/main/generated/torchaudio.datasets.LibriLightLimited.html), and return a **CSV file** containing the Semantic, Coarse and Fine tokens. 

During dataset creation, we crop our training data in pieces, as suggested by the paper, but we used **halved durations** (segments of 15, 5, 1 seconds respectively for Semantic, Coarse and Fine transformers). We also provided two version of dataset creation: *storeTokens(...)* allows to use local audio data, while  *store_from_librilight(...)* uses the before mentioned Torch dataset. We used the second version for training our model.

In [26]:
tokenPath = "out" ## output file directory
tokenFile = "tokens.csv" ## output file name
audioPath = "data" ## data_location

In [ ]:
#fileCount = storeTokens(audioPath, tokenPath, tokenFile, w2vBERT, soundStream, fileCountCheckpoint = 10)
fileCount = store_from_librilight(tokenPath, tokenFile, w2vBERT, soundStream, fileCountCheckpoint = 10, subset = "10h")

In [8]:
AUDIO_LENGTH = 30
CROP_LENGTH = [15,5,1] #Original is [30, 10, 3]
USE_OFFSET = False
TRAINING_PERCENTAGE = 0.8
BATCH_SIZE = 16
MAX_EPOCH = 30

common_params = {
	'tokenPath' : tokenPath, 
	'tokenFile' : tokenFile, 
	'expected_audio_length' : 30, 
	'crop_length' : CROP_LENGTH, 
	'useOffset' : USE_OFFSET, 
	'training_percentage' : TRAINING_PERCENTAGE, 
	'batch_size' : BATCH_SIZE
}

## AudioLM: a transformer-based audio model

Once we have converted our data into token sequences, we can start defining the **generator** model. AudioLM network is based on three Decoder-only transformers, each of them dedicated to the **auto-regressive** generation of a specific kind of token. 
During inference, we first generate the new semantic tokens, and then use them to condition the generation of new acoustic tokens. With this structure, we can safely assume that *semantic tokens are expected to be conditionally independent from past acoustic tokens* given past semantic tokens:
$$
  p(z_{t}|z_{\lt t},y_{\lt t}) \simeq p(z_{t}|z_{\lt t})
$$



### Some implementation details

In the original paper, each generator follows **the same identical decoder-only Transformer** structure, with:
- 12 decoder layers
- 16 attention heads
- 1024 as embedding dimension
- 4096 as dimension for the feed-forward layer
- 0.1 as dropout value

The original model is also enriched with a **T5-style relative positional encoding**, that is trained along with the three stage transformers. For this reimplementation, we adapted an implementation of the paper *"Self-Attention with Relative Position Representations"* provided by https://github.com/AliHaiderAhmad001/Self-Attention-with-Relative-Position-Representations. We also reduced the size of the model for computational reasons, as it will be shown later. In general, we tried to reconstruct the original structure of the Decoders, considering that the **original model is closed-source**.

In [9]:
model_params = {
	'd_model': 256, #1024
	'num_layers': 3, #12
	'num_heads': 4, #16
	'dim_feedforward': 1024, #4096
	'dropout': 0.1,
	'learning_rate': 10e-4,
	'myDevice': torch.device("cuda" if torch.cuda.is_available() else "cpu"),
	'Q_prime' : 3,
	'Q' : 8
}

#N.B. The semantic vocab_size should always be 500, the other two should be 1024 * Q_prime and 1024 * Q respectively if we use the offset, 1024 if we don't
if USE_OFFSET:
	VOCAB_SIZE = [500, 1024 * model_params['Q_prime'], 1024 * model_params['Q']]
else:
	VOCAB_SIZE = [500, 1024, 1024]

### Semantic Transformer: expanding a sentence snippet

<center><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/semantic.png"/></center>

The first transformer is responsible for the generation of new semantic tokens, estimating $ p(z_{t}|z_{\lt t}) $, where $ z_{i} $ denotes the $i$-th semantic token. In this way, given the semantic aspects of the prompt, the network is able to generate **new coherent content**, keeping long-term consistency.

In [ ]:
semantic_train_dataloader, semantic_valid_dataloader = getSingleModelDataLoaders(mode = "semantic", **common_params)

semantic_trainer = setSingleTrainer(max_epoch = MAX_EPOCH)

In [ ]:
semantic_checkpoint = "checkpoints/semantic-checkpoint-10h-colab.ckpt"

semantic_model = createSingleModel(type = "semantic", checkpoint_path = semantic_checkpoint, **model_params, audioDuration = CROP_LENGTH[0], vocab_size = VOCAB_SIZE[0])

In [ ]:
modelFit(model = semantic_model, trainer = semantic_trainer, train_loader = semantic_train_dataloader, valid_loader = semantic_valid_dataloader, checkpoint_path = semantic_checkpoint, myDevice=model_params['myDevice'])

### Coarse Transformer: generating new audio

<center><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/coarse.png"/></center>

Once we have defined a mechanism to generate new semantic content, we need to generate the corresponding acoustic content. Since we want the new tokens to be **coherent both with the sentence meaning** (represented by the Semantic tokens) and the **original speaker and enviroment conditions** (represented by the past Coarse Tokens), this model is trained to estimate:
$$ p(y_t^q \mid z, y_{<t}^{<Q'}, y_t^{<q})  \quad \text{for} \ q \le Q'$$
where the current generated semantic token depends on **Semantic tokens** $z$ and the **acoustic tokens** $ y_{<t}^{<Q'} $ and $ y_t^{<q} $ that references respectively to **previous and current audio parts**. In the formula, $Q'$ denotes the **number of Soundstream quantizer** dedicated to Coarse tokens generation.


In [ ]:
coarse_train_dataloader, coarse_valid_dataloader = getSingleModelDataLoaders(mode = "coarse", **common_params)

coarse_trainer = setSingleTrainer(max_epoch = MAX_EPOCH)

In [ ]:
coarse_checkpoint = "checkpoints/coarse-checkpoint-10h-colab.ckpt"

coarse_model = createSingleModel(type = "coarse", checkpoint_path = coarse_checkpoint, **model_params, audioDuration = CROP_LENGTH[1], vocab_size=VOCAB_SIZE[1])

In [ ]:
modelFit(model = coarse_model, trainer = coarse_trainer, train_loader = coarse_train_dataloader, valid_loader = coarse_valid_dataloader, checkpoint_path = coarse_checkpoint, myDevice=model_params['myDevice'])

### Fine Transformer: rebuilding a detailed audio waveform

<center><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/fine.png"/></center>

At last, we need to generate the **acoustic details** of the extended audio, since we need to recreate the **hierarchical structure** of acoustic features needed for the SoundStream model. This is easily done through a third and last transformer, that given the Coarse tokens and the previous Fine tokens, is trained to generate new Fine tokens. In formula:

$$ p(y_t^q \mid y_{<t}^{\leq Q'}, y_{\geq t}^{> Q'}, y_t^{<q}) \quad \text{for} \ q > Q' $$

where the new tokens depends on the **Coarse tokens** $y_{<t}^{\leq Q'}$ and the **Fine tokens** $y_{\geq t}^{> Q'}$ and $y_t^{<q}$, belonging respectively to **previous and current audio parts**.

In [ ]:
fine_train_dataloader, fine_valid_dataloader = getSingleModelDataLoaders(mode = "fine", **common_params)
fine_trainer = setSingleTrainer(max_epoch = MAX_EPOCH)

In [ ]:
fine_checkpoint = "checkpoints/fine-checkpoint-10h-colab.ckpt"

fine_model = createSingleModel(type = "fine", checkpoint_path = fine_checkpoint, **model_params, audioDuration = CROP_LENGTH[2], vocab_size = VOCAB_SIZE[2])

In [ ]:
modelFit(model = fine_model, trainer = fine_trainer, train_loader = fine_train_dataloader, valid_loader = fine_valid_dataloader, checkpoint_path = fine_checkpoint, myDevice=model_params['myDevice'])

## Inference and results

### How to extend an audio input

After the training we can try use our models. First of all we prepare a single audio input using Hubert and SoundStream and then, through the AudioLM model, we can **generate a new sequence** with our preferred length.

In [22]:
semantic_tokens, coarse_tokens, fine_tokens = prepare_single_audio("exampleAudio.flac", w2vBERT, soundStream, audioDuration=2, useOffset = USE_OFFSET)

In [ ]:
generated_acoustic_tokens = generate_new_sequence(semantic_tokens, coarse_tokens, fine_tokens, semantic_model, coarse_model, fine_model, audioDuration=5)

Once we have generated both Coarse acoustic tokens and Fine acoustic tokens, we can **reshape them into a format that resembles the original data** provided by the Residual Vector Quantizer component of SoundStream. Finally we pass the Acoustic tokens into the Decoder part of Soundstream, that will return the **final audio waveform**.

<center><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/full.png"/></center>

In [24]:
waveform = tokens_to_audio(coarse_tokens=generated_acoustic_tokens[0], fine_tokens=generated_acoustic_tokens[1], model=soundStream, removeOffsets=USE_OFFSET)

In [ ]:
print("The extended audio:\n")
display(Audio(waveform, rate = 16000))

### Training results

Because of **prohibiting computational time and lack of resources**, we tried to move the training from local CPU to Colab. We noticed that, even though the **training time has decreased by a lot**, the memory limit imposed by **Colab didn't allow to load the entire model**.

We opted for reducing the structure of the model, changing the number of layers and parameters. We worked on two version of the model: the first one uses about a **quarter** of the original parameters values, while the second one uses **halfed** values for the parameters. Using the full model caused a **"Out of Memory error"** in Colab.

For the same reason, **the training of the model over 60000 hours of data, as suggested by the paper, was not feasible**, both for time and memory constraints given by Colab service. So we decided to make some tests on both 1 hour and 10 hours of data, and analyse the difference between the two approaches.

For the same reasons, only the Coarse Transformer has been trained. In fact, while training three transformers has been proved unfeasible, we considered the Coarse Transformer as the best candidate to train because:

- With respect to the Semantic Transformer, it uses a condition input sequence;
- With respect to the Fine Transformer, the size of the vocabulary and the input sequence length were more manageable.
- Inference on the Coarse Transformer gives the more interpretable results, giving a coarse but comprehensible audio result.

Every test shown below is the result obtained by 30 epochs of training.

Here we show some tests on 1 hour of data. We tried **three different cases**:

- A model that uses both the offsets for Acoustic Tokens (increasing the vocab_size parameter) and a dual embedding for conditioning and generator tokens;
- A model that uses only the offsets, without dual embedding
- A model that uses neither of them

As seen below, the reduced Quarter-model responds well on the most basic alternative, while the Halfed-model appears to get some benefits from using offsets on acoustic tokens. Dual embedding usage doesn't seem to provide any advantage, at least while tested in these two conditions.

<table>
  <tr>
    <th><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/Q-1a.svg"/></th>
    <th><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/H-1a.svg"/></th>
    
  </tr>
  <tr>
    <th><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/Q-1b.svg"/></th>
    <th><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/H-1b.svg"/></th>
  </tr>
    <tr>
        <th><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/Q-1c.svg"/></th>
        <th><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/H-1c.svg"/></th>
    </tr>
</table> 

At last, we report the results obtained with **10 hours of data**. As we can see, both the **accuracy and validation loss have improved** with the increased amount of used data. This suggests that, since the model uses a Transformer based architecture, the **amount of data affects significantly the generator performance**. It's also interesting to observe that changing the number of parameters doesn't seem to particularly affect the validation loss and accuracy.

<table>
  <tr>
    <th><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/Q-10a.svg"/></th>
    <th><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/H-10a.svg"/></th>
    
  </tr>
  <tr>
    <th><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/Q-10b.svg"/></th>
    <th><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/H-10b.svg"/></th>
  </tr>
    <tr>
        <th><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/Q-10c.svg"/></th>
        <th><img src="https://raw.githubusercontent.com/GitGab-dev/NN_Project-AudioLM/main/reportImg/H-10c.svg"/></th>
    </tr>
</table>

## References

- AudioLM paper: https://arxiv.org/abs/2209.03143
- AudioLM blog post: https://research.google/blog/audiolm-a-language-modeling-approach-to-audio-generation/
- HuBERT and K-Means implementation: https://github.com/facebookresearch/fairseq/blob/main/examples/hubert
- w2v-BERT paper: https://arxiv.org/abs/2108.06209
- SoundStream implementation: https://github.com/kaiidams/soundstream-pytorch
- SoundStream paper: https://arxiv.org/abs/2107.03312
- LibriLight full dataset: https://github.com/facebookresearch/libri-light/tree/main
- LibriLight Limited on pyTorch: https://pytorch.org/audio/main/generated/torchaudio.datasets.LibriLightLimited.html
- Relative positional embeddings implementation: https://github.com/AliHaiderAhmad001/Self-Attention-with-Relative-Position-Representations
  